In [1]:
USER_FLAG = "--user"

In [2]:
!python -m pip install --upgrade pip --user
!pip install example --use-feature=2020-resolver

In [2]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.0.0 --upgrade
!pip3 install {USER_FLAG} kfp google-cloud-pipeline-components==0.1.1 --upgrade

In [ ]:
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [3]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.10
google_cloud_pipeline_components version: 0.1.1


#### PROJECT_ID AND BUCKET

In [4]:
import os
PROJECT_ID = ""
# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  kedro-kubeflow-334417


In [5]:
BUCKET_NAME="gs://diab-gsbucket"

In [ ]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import storage
import os
from kfp.v2 import dsl, compiler
from kfp.v2.google.client import AIPlatformClient
import google_cloud_pipeline_components.aiplatform as gcc_ai

#### CONSTANTS

In [6]:
from typing import NamedTuple
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import storage
import os
import pandas as pd
import datetime
from kfp.v2 import dsl, compiler
from kfp.v2.google.client import AIPlatformClient
import google_cloud_pipeline_components.aiplatform as gcc_ai
from typing import NamedTuple
import kfp
from kfp import dsl
import kfp.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import storage
from io import StringIO
import pandas as pd
from google.cloud import storage
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [8]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"
PIPELINE_NAME = "diabetes_pipeline"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://diab-gsbucket/pipeline_root/'

#### COMPONENTS

##### 1. READ DATASET

In [11]:
'''
@component(base_image="gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest",
            output_component_file="read_data_component.yaml")
def read_data(bucket: str = BUCKET_NAME,
             path: str = "data",
             file: str = "diabetes"):
    df = pd.read_csv(f'gs://{bucket}/{path}/{file}.csv', sep=",")
    '''

'\n@component(base_image="gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest",\n            output_component_file="read_data_component.yaml")\ndef read_data(bucket: str = BUCKET_NAME,\n             path: str = "data",\n             file: str = "diabetes"):\n    df = pd.read_csv(f\'gs://{bucket}/{path}/{file}.csv\', sep=",")\n    '

##### 1. READ  + TEST/TRAIN DATA SPLIT

In [67]:
@component(base_image="google/cloud-sdk:279.0.0",
            output_component_file="test_train_data_split_component.yaml")
def data_component(bucket: str, path: str, data_file: str, value: float, marker: int) -> int:
    
    import pandas as pd
    import sklearn
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import kfp
    from kfp import dsl
    from kfp.v2 import compiler
    from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                            OutputPath, ClassificationMetrics, Metrics, component)
    from kfp.v2.google.client import AIPlatformClient
    from google.cloud import aiplatform
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    from google.cloud import storage
    import os
    from kfp.v2 import dsl, compiler
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    
    data = pd.read_csv(f'gs://{bucket}/{path}/{data_file}.csv', sep=",")
    train_data = data.drop('Outcome',axis=1)
    test_data = data['Outcome']
    
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_data, test_data, test_size = value, random_state=42)
    
    # upload train/test data innto cloud storage
    client = storage.Client()
    bucket = client.get_bucket(f'{bucket}')
    
    # 1. X_train
    bucket.blob('data/X_train.csv').upload_from_string(X_train.to_csv(), 'text/csv')
    xtrain = f"gs://{bucket}/{path}/X_train.csv"
    # 2. X_test
    bucket.blob('data/X_test.csv').upload_from_string(X_test.to_csv(), 'text/csv')
    xtest = f"gs://{bucket}/{path}/X_test.csv"
    # 3. y_train
    bucket.blob('data/y_train.csv').upload_from_string(y_train.to_csv(), 'text/csv')
    ytrain = f"gs://{bucket}/{path}/y_train.csv"
    # 4. y_test
    bucket.blob('data/y_test.csv').upload_from_string(y_test.to_csv(), 'text/csv')
    ytest = f"gs://{bucket}/{path}/y_test.csv"
    
    marker = 1
    return marker

##### 2. TRAIN + VALIDATE MODEL

In [68]:
@component(base_image="google/cloud-sdk:279.0.0",
            output_component_file="model_component.yaml")
def model_component(bucket:str, path:str, xtrain:str, ytrain:str, xtest:str, ytest:str, marker: int) -> float:
    
    import pandas as pd
    import sklearn
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import kfp
    from kfp import dsl
    from kfp.v2 import compiler
    from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                            OutputPath, ClassificationMetrics, Metrics, component)
    from kfp.v2.google.client import AIPlatformClient
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    from google.cloud import storage
    import os
    from kfp.v2 import dsl, compiler
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    
    with dsl.Condition(marker == 1):
        # read X_train and y_train from cloud storage
        X_train = pd.read_csv(f'gs://{bucket}/{path}/{xtrain}.csv', sep=",")
        y_train = pd.read_csv(f'gs://{bucket}/{path}/{ytrain}.csv', sep=",")
        # train model
        rc = RandomForestClassifier(max_depth=2, random_state=0)
        model = rc.fit(X_train, y_train)
    
        # read X_test and y_test from cloud storage
        X_test = pd.read_csv(f'gs://{bucket}/{path}/{xtest}.csv', sep=",")
        y_test = pd.read_csv(f'gs://{bucket}/{path}/{ytest}.csv', sep=",")
        # validate model
        y_pred = rc.predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)
        
    return accuracy

In [69]:
@component(base_image="google/cloud-sdk:279.0.0",
            output_component_file="decision_component.yaml")
def decision_component(accuracy:float) -> None:
    print(accuracy)

In [70]:
@component(base_image="google/cloud-sdk:279.0.0",
            output_component_file="true_decision_component.yaml")
def true_component(accuracy:float) -> None:
    print(f'Yes!! {accuracy} is the Accuracy and its greater than the threshold')

In [71]:
@component(base_image="google/cloud-sdk:279.0.0",
            output_component_file="false_decision_component.yaml")
def false_component(accuracy:float) -> None:
    print(f'No. {accuracy} is the Accuracy and its smaller than the threshold')

#### PIPELINE

In [72]:
@kfp.dsl.pipeline(name = "diabetes-pipeline",
                  pipeline_root = PIPELINE_ROOT)
def diab_pipeline(
    display_name: str=f"{PIPELINE_NAME}-2021MMDD-nn",
    project: str = PROJECT_ID,
    gcp_region: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    marker: int = 0,
    test_train_split_ratio: float = 0.3,
    accuracy_threshold: float = 0.5
) -> None:
    
    import pandas as pd
    import sklearn
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import kfp
    from kfp import dsl
    from kfp.v2 import compiler
    from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                            OutputPath, ClassificationMetrics, Metrics, component)
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    from google.cloud import storage
    import os
    from kfp.v2 import dsl, compiler
    from kfp.v2.google.client import AIPlatformClient
    import google_cloud_pipeline_components.aiplatform as gcc_ai
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    
    # initiating data component
    data_op = data_component(BUCKET_NAME, "data", "diabetes", test_train_split_ratio, marker)
    # initiating model component
    model_op = model_component(BUCKET_NAME, "data", "X_train", "y_train", "X_test", "y_test", data_op.output)
    
    with dsl.Condition(model_op.output >= accuracy_threshold, name="accuracy>=50"):
        true_component(model_op.output)
    with dsl.Condition(model_op.output < accuracy_threshold, name="accuracy<50"):
        false_component(model_op.output)
    '''
    # decision node
    with kfp.dsl.Condition(acc_metric.output >= accuracy_threshold, name="accuracy check"):
        print(f"{acc_metric} is accuracy and is greater than its threshold")
    '''
    

#### COMPILE + RUN

In [73]:
compiler.Compiler().compile(
    pipeline_func=diab_pipeline, package_path="diab_pipeline.json"
)

In [74]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

In [75]:
response = api_client.create_run_from_job_spec(
    "diab_pipeline.json", pipeline_root=PIPELINE_ROOT,
)

In [18]:
!pip install google-cloud-aiplatform

### DOCKER IMAGE

In [99]:
!sudo usermod -a -G docker ${USER}

In [100]:
!docker run busybox date

Unable to find image 'busybox:latest' locally
latest: Pulling from library/busybox

4ad355aa: Pull complete 2.8kB/772.8kBBDigest: sha256:5acba83a746c7608ed544dc1533b87c737a0b0fb730301639a0179f9344b1678
Status: Downloaded newer image for busybox:latest
Fri Jan  7 10:24:00 UTC 2022


In [102]:
# already done by default
# !gcloud auth configure-docker

In [ ]:
!vim Dockerfile

7="Dockerfile" [New File]▽  [>c]10;?]11;?~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                       

In [115]:
!export IMAGE_REPO_NAME=vertexai_repo
!export IMAGE_TAG=img_tag
!export IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_REPO_NAME:$IMAGE_TAG

!docker build -t Dockerfile .

invalid argument "Dockerfile" for "-t, --tag" flag: invalid reference format: repository name must be lowercase
See 'docker build --help'.


In [104]:
!docker run $IMAGE_URI --epochs 1

unknown flag: --epochs
See 'docker run --help'.


In [105]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED         SIZE
busybox                        latest    beae173ccac6   7 days ago      1.24MB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   10 months ago   1.73GB


In [106]:
!docker build --help


Usage:  docker build [OPTIONS] PATH | URL | -

Build an image from a Dockerfile

Options:
      --add-host list           Add a custom host-to-IP mapping (host:ip)
      --build-arg list          Set build-time variables
      --cache-from strings      Images to consider as cache sources
      --cgroup-parent string    Optional parent cgroup for the container
      --compress                Compress the build context using gzip
      --cpu-period int          Limit the CPU CFS (Completely Fair
                                Scheduler) period
      --cpu-quota int           Limit the CPU CFS (Completely Fair
                                Scheduler) quota
  -c, --cpu-shares int          CPU shares (relative weight)
      --cpuset-cpus string      CPUs in which to allow execution (0-3, 0,1)
      --cpuset-mems string      MEMs in which to allow execution (0-3, 0,1)
      --disable-content-trust   Skip image verification (default true)
  -f, --file string             Name of the Docker

## ROUGH SPACE

In [80]:
!pip-compile -v

/bin/bash: pip-compile: command not found


In [78]:
!sudo apt-get install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (18.1-5).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [97]:
!pipdeptree -f > requirements.txt

Warning!!! Possibly conflicting dependencies found:
* apache-beam==2.34.0
 - pyarrow [required: >=0.15.1,<6.0.0, installed: 6.0.1]
 - dill [required: >=0.3.1.1,<0.3.2, installed: 0.3.4]
 - httplib2 [required: >=0.8,<0.20.0, installed: 0.20.2]
* cloud-tpu-client==0.10
 - google-api-python-client [required: ==1.8.0, installed: 1.12.8]
* explainable-ai-sdk==1.3.2
 - xai-image-widget [required: Any, installed: ?]
* pandas-profiling==3.0.0
 - tangled-up-in-unicode [required: ==0.1.0, installed: 0.2.0]
* tensorflow-io==0.21.0
 - tensorflow-io-gcs-filesystem [required: ==0.21.0, installed: 0.23.1]
 - tensorflow [required: >=2.6.0,<2.7.0, installed: 2.7.0]
* tensorflow-transform==1.5.0
 - pyarrow [required: >=1,<6, installed: 6.0.1]
* tfx-bsl==1.5.0
 - pyarrow [required: >=1,<6, installed: 6.0.1]
------------------------------------------------------------------------


In [95]:
!pipdeptree -f --warn silence | grep -E '^[a-zA-Z0-9\-]+'

appdirs==1.4.4
asn1crypto==1.4.0
async-generator==1.10
backports.functools-lru-cache==1.6.4
beatrix-jupyterlab==3.1.6
blinker==1.4
brotlipy==0.7.0
cloud-tpu-client==0.10
cloud-tpu-profiler==2.4.0
conda==4.11.0
conda-package-handling==1.7.3
confuse==1.7.0
cookiecutter==1.7.3
dataclasses==0.8
docker-pycreds==0.4.0
example==0.1.0
explainable-ai-sdk==1.3.2
explainers==0.1
gcsfs==2021.11.1
google-apitools==0.5.31
google-cloud-bigtable==2.4.0
google-cloud-dataproc==3.1.1
google-cloud-datastore==2.4.0
google-cloud-dlp==1.0.0
google-cloud-firestore==2.3.4
google-cloud-kms==2.10.1
google-cloud-language==2.3.1
google-cloud-logging==2.7.0
google-cloud-monitoring==2.8.0
google-cloud-pipeline-components==0.1.1
google-cloud-pubsub==1.7.0
google-cloud-recommendations-ai==0.2.0
google-cloud-scheduler==2.5.0
google-cloud-spanner==3.12.0
google-cloud-speech==2.11.1
google-cloud-tasks==2.7.1
google-cloud-translate==3.6.1
google-cloud-videointelligence==2.5.1
google-cloud-vision==2.6.3
grpcio-gcp==0.2.2
g

In [94]:
!pipdeptree --warn silence | grep -E '^\w+'

appdirs==1.4.4
asn1crypto==1.4.0
async-generator==1.10
backports.functools-lru-cache==1.6.4
beatrix-jupyterlab==3.1.6
blinker==1.4
brotlipy==0.7.0
cloud-tpu-client==0.10
cloud-tpu-profiler==2.4.0
conda==4.11.0
conda-package-handling==1.7.3
confuse==1.7.0
cookiecutter==1.7.3
dataclasses==0.8
docker-pycreds==0.4.0
example==0.1.0
explainable-ai-sdk==1.3.2
explainers==0.1
gcsfs==2021.11.1
google-apitools==0.5.31
google-cloud-bigtable==2.4.0
google-cloud-dataproc==3.1.1
google-cloud-datastore==2.4.0
google-cloud-dlp==1.0.0
google-cloud-firestore==2.3.4
google-cloud-kms==2.10.1
google-cloud-language==2.3.1
google-cloud-logging==2.7.0
google-cloud-monitoring==2.8.0
google-cloud-pipeline-components==0.1.1
google-cloud-pubsub==1.7.0
google-cloud-recommendations-ai==0.2.0
google-cloud-scheduler==2.5.0
google-cloud-spanner==3.12.0
google-cloud-speech==2.11.1
google-cloud-tasks==2.7.1
google-cloud-translate==3.6.1
google-cloud-videointelligence==2.5.1
google-cloud-vision==2.6.3
grpcio-gcp==0.2.2
g

In [91]:
!pipdeptree

Warning!!! Possibly conflicting dependencies found:
* apache-beam==2.34.0
 - dill [required: >=0.3.1.1,<0.3.2, installed: 0.3.4]
 - httplib2 [required: >=0.8,<0.20.0, installed: 0.20.2]
 - pyarrow [required: >=0.15.1,<6.0.0, installed: 6.0.1]
* cloud-tpu-client==0.10
 - google-api-python-client [required: ==1.8.0, installed: 1.12.8]
* explainable-ai-sdk==1.3.2
 - xai-image-widget [required: Any, installed: ?]
* pandas-profiling==3.0.0
 - tangled-up-in-unicode [required: ==0.1.0, installed: 0.2.0]
* tensorflow-io==0.21.0
 - tensorflow-io-gcs-filesystem [required: ==0.21.0, installed: 0.23.1]
 - tensorflow [required: >=2.6.0,<2.7.0, installed: 2.7.0]
* tensorflow-transform==1.5.0
 - pyarrow [required: >=1,<6, installed: 6.0.1]
* tfx-bsl==1.5.0
 - pyarrow [required: >=1,<6, installed: 6.0.1]
------------------------------------------------------------------------
appdirs==1.4.4
asn1crypto==1.4.0
async-generator==1.10
backports.functools-lru-cache==1.6.4
beatrix-jupyterlab==3.1.6
  - googl

In [83]:
!python -m pip


Usage:   
  /opt/conda/bin/python -m pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper command used for command completion.
  

In [84]:
!python -m pip list

Package                               Version
------------------------------------- ------------
absl-py                               0.11.0
aiohttp                               3.8.1
aiosignal                             1.2.0
ansiwrap                              0.8.4
anyio                                 3.4.0
apache-beam                           2.34.0
appdirs                               1.4.4
argcomplete                           1.12.3
argon2-cffi                           21.1.0
arrow                                 1.2.1
asn1crypto                            1.4.0
astunparse                            1.6.3
async-generator                       1.10
async-timeout                         4.0.1
asynctest                             0.13.0
attrs                                 21.2.0
avro-python3                          1.9.2.1
Babel                                 2.9.1
backcall                              0.2.0
backports.entry-points-selectable     1.1.1
backports.functo

In [82]:
!python -m pip freeze 

absl-py==0.11.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1637087016127/work
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1636093929600/work
ansiwrap==0.8.4
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1637703203932/work/dist
apache-beam==2.34.0
appdirs @ file:///home/conda/feedstock_root/build_artifacts/appdirs_1603108395799/work
argcomplete @ file:///home/conda/feedstock_root/build_artifacts/argcomplete_1619128689661/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1636021379047/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1635261338479/work
asn1crypto @ file:///home/conda/feedstock_root/build_artifacts/asn1crypto_1595949944546/work
astunparse==1.6.3
async-generator==1.10
async-timeout @ file:///home/conda/feedstock_root/build_artifacts/async-timeout_1637092647930/work
asynctest==0.13.0
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_16203879